In [4]:
import tensorflow as tf
import tensorflow.keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
import tensorflow.keras.datasets as datasets
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable = True)

In [13]:
(train, test), metadata = tfds.load(
    "cifar10",
    split = ["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True,
)

def normalize(image, label):
    return tf.cast(image, tf.float32)/255.0, label

def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))
    
    if (tf.random.uniform((), minval = 0, maxval = 1) < 0.1):
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])
    
    image = tf.image.random_brightness(image, max_delta = 0.1)
    image = tf.image.random_contrast(image, lower = 0.1, upper = 0.2)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)

    return image, label

BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE
train = train.map(normalize, num_parallel_calls = AUTOTUNE).cache()
train = train.shuffle(metadata.splits["train"].num_examples)
# train = train.map(augment, num_parallel_calls = AUTOTUNE)
train = train.batch(BATCH_SIZE)
train = train.prefetch(AUTOTUNE)

test = test.map(normalize, num_parallel_calls = AUTOTUNE).cache()
test = test.batch(BATCH_SIZE)
test = test.prefetch(AUTOTUNE)

In [15]:
new_height = new_width =32
data_augmentation = Sequential([
    layers.experimental.preprocessing.Resizing(height = new_height, width = new_width),
    layers.experimental.preprocessing.RandomFlip(mode = "horizontal"),
    layers.experimental.preprocessing.RandomContrast(factor = 0.1),
])

inp = keras.Input(shape = (32, 32, 3))
data_augment = data_augmentation(inp)
conv1 = layers.Conv2D(4, 3, padding = 'same', activation = 'relu')(inp)
conv2 = layers.Conv2D(8, 3, padding = 'same', activation = 'relu')(conv1)
pool = layers.MaxPool2D()(conv2)
conv3 = layers.Conv2D(16, 3, activation = 'relu', padding = 'same')(pool)
flatten = layers.Flatten()(conv3)
dense1 = layers.Dense(64, activation = 'relu')(flatten)
output = layers.Dense(10)(dense1)
model = Model(inputs = [inp, ], outputs = [output, ])
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_6 (Conv2D)           (None, 32, 32, 4)         112       
                                                                 
 conv2d_7 (Conv2D)           (None, 32, 32, 8)         296       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 8)        0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 16, 16, 16)        1168      
                                                                 
 flatten_2 (Flatten)         (None, 4096)              0         
                                                           

In [16]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy'],
)

In [ ]:
save_callback = keras.callbacks.ModelCheckpoint(
    'checkpoints/',
    save_weights_only = True,
    monitor = 'accuracy',
    save_best_only = False,
)

custom_callback = lambda epoch, lr: lr if epoch<2 else lr*0.99
lr_scheduler = keras.callbacks.LearningRateScheduler(custom_callback)

# Completely Custom Callback:
class CustomLR(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        old_lr = float(tf.keras.backend.get_value(self.model.optimizer.lr))
        if (epoch < 2):
            new_lr = old_lr
        else:
            new_lr = old_lr*0.99

        keras.backend.set_value(self.model.optimizer, new_lr)
        print(f"LR: {new_lr} | Epoch: {epoch+1}")


In [17]:
# model.fit(train, epochs = 5, verbose = 2, callbacks = [save_callback, lr_scheduler, CustomLR()])
model.fit(train, epochs = 5, verbose = 2)
model.evaluate(test, verbose = 2)

Epoch 1/5
1563/1563 - 10s - loss: 1.4856 - accuracy: 0.4735 - 10s/epoch - 6ms/step
Epoch 2/5
1563/1563 - 9s - loss: 1.1414 - accuracy: 0.5998 - 9s/epoch - 6ms/step
Epoch 3/5
1563/1563 - 11s - loss: 1.0151 - accuracy: 0.6479 - 11s/epoch - 7ms/step
Epoch 4/5
1563/1563 - 8s - loss: 0.9277 - accuracy: 0.6768 - 8s/epoch - 5ms/step
Epoch 5/5
1563/1563 - 8s - loss: 0.8511 - accuracy: 0.7025 - 8s/epoch - 5ms/step
313/313 - 2s - loss: 1.0544 - accuracy: 0.6323 - 2s/epoch - 5ms/step


[1.0543687343597412, 0.6323000192642212]